<a href="https://colab.research.google.com/github/pnkjjha/BuildNExperiment/blob/main/googlecolab_Spark_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=40328d043694c6cb3a222dbca37c9fa55fca7713cf76bfc4caa2201d37ea7e8f
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
import pyspark

In [6]:
import requests

In [4]:
spark = pyspark.sql.SparkSession.builder.appName('practice').getOrCreate()

In [8]:
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
response = requests.get(url, stream=True)
with open("yellow_tripdata_2024-01.parquet", "wb") as f:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)



In [14]:
# Read the Parquet file from the local file system
df1 = spark.read.parquet("yellow_tripdata_2024-01.parquet")

In [10]:
df1.show()

In [15]:
df1.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [16]:
df1.count()

2964624

In [24]:
df1.first()

Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 1, 1, 0, 57, 55), tpep_dropoff_datetime=datetime.datetime(2024, 1, 1, 1, 17, 43), passenger_count=1, trip_distance=1.72, RatecodeID=1, store_and_fwd_flag='N', PULocationID=186, DOLocationID=79, payment_type=2, fare_amount=17.7, extra=1.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=22.7, congestion_surcharge=2.5, Airport_fee=0.0)

In [28]:
df1.filter(df1.VendorID == 1).count()

729732

In [32]:
df1.groupBy('VendorID').count().show()

+--------+-------+
|VendorID|  count|
+--------+-------+
|       1| 729732|
|       2|2234632|
|       6|    260|
+--------+-------+



when mentioning groupy you dont have to mention select because they are always same set of columns.

In [36]:
df1.groupBy('VendorID').sum('tip_amount').orderBy('VendorID', ascending=False).show()

+--------+------------------+
|VendorID|   sum(tip_amount)|
+--------+------------------+
|       6|               0.0|
|       2| 7700953.170000889|
|       1|2188647.1400000826|
+--------+------------------+

